# Preprocessing


Fluorescence datasets ideally reflect a relationship between the pixels in an image and the location and local density of your fluorescent molecule in a sample. However, properties of the detectors, optics, or even the samples can confound direct interpretation of this data. Here we will present some operations that can mitigate these effects to achieve robust hypothesis testing. 

Today, we will define the ROIs (regions of interest) in this set of images. You may have noticed some qualities of the above images that make them hard to reliably threshold into masks relecting the relevant cell organelles.

Part 1 Getting Ready
1. Loading images
2. Metadata
3. Visual inspection


Part 2: Global Threshold
1. Difference in intensity between images.
    - Introduction to automated thresholding methods.

Part 3: Local Threshold
1. Noise corrupting the images
    - Introduction to Rank Filters: median filter
2. Uneven illumination in the nucleus channel makes finding a single threshold across a single image challenging. 
    - Rolling ball background subtraction
3. Uneven paxillin expression, low EGFP signal, and cytoplasmic signal complicate focal adhesion thresholding
    - thresholding within defined ROIs 

First, let's import packages.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
import skimage

sns.set_style('dark', rc={'image.cmap':'inferno'})

First, check files in the directory.

In [ ]:
# CHANGE PATH TO YOUR PC WHERE DATA IS
data_path = "/Users/ahmedelkaffas/Documents/Repositories/DataLucense/dev-summer2018/Data_ConfocalDrugPanel/"
import os
os.listdir(data_path)
print(data_path)

Then, read files of interest, in our case, dataset with and without drug. Check dimensions of the dataset.

In [ ]:
from skimage.io import imread

data_drug = np.uint16(imread(data_path + "drugA.tif")) #treated with drugA
data_nodrug = np.uint16(imread(data_path + "DMSO.tif")) #treated without drug

In [ ]:
data_drug.shape

Let's examine the metadata- json is the format for this dataset.

In [ ]:
import json
with open( data_path + 'DMSO_metadata.json', mode='r') as f_nodrug:
    meta_nodrug = json.load(f_nodrug)

In [ ]:
meta_nodrug 
#or more nicely, 
#for key, value in meta_nodrug.items():
#    print(key)

Let's re-label the slices with names, instead of numbers.

In [ ]:
drug_slice = {}
nodrug_slice = {}
for idx, channel in enumerate(meta_nodrug['channels']):
    drug_slice[channel] = data_drug[3,:,:,idx]
    nodrug_slice[channel] = data_nodrug[3,:,:,idx] #add in the indexing when read in full dataset
    print(channel)

We are ready to plot and visualize our dataset! We can see there is a change before drug vs after drug.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(nodrug_slice["actin"])
ax[1].imshow(nodrug_slice['nucleus'])
ax[2].imshow(nodrug_slice["your_fav_protein"])
fig.suptitle('Before Drug')

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(drug_slice["actin"])
ax[1].imshow(drug_slice['nucleus'])
ax[2].imshow(drug_slice["your_fav_protein"])
fig.suptitle('After Drug')

Now, let's start pre-processing our dataset. We will focus on the "your_fav_protein" dataset.

In [ ]:
data1 = nodrug_slice["your_fav_protein"]
data2 = drug_slice["your_fav_protein"]

First, let's try global threshold, selecting one, single value for the entire data set. The widget allows to change the value manually.

In [ ]:
#data1 = vinc["nucleus"]
#data2 = wt["nucleus"]

from ipywidgets import interactive
@interactive
def show_masks(thresh=(0, data1.max() * 0.3, 500)): #shwo_masks a function, whey thresh have 3 input
    fig, ax = plt.subplots(1, 2, figsize=(64, 50))
    
    mask1 = np.zeros(data1.shape)
    mask2 = np.zeros(data2.shape)
    
    mask1[data1 >=thresh] = 1
    mask2[data2 >=thresh] = 1
    
    ax[0].imshow(mask1, vmin=0, vmax=1)
    ax[1].imshow(mask2, vmin=0, vmax=1)
show_masks

It looks like thresholding fails- a ring like structure is visible. There might be an uneven illumination and other noises in the dataset. Let's filter out these noises. Here's our raw data: can you see some of noises? Speckles are visible. Let's remove these 'salt-and-pepper' noise first.

In [ ]:
plt.imshow(nodrug_slice["your_fav_protein"])

We need to import packages and filters. (Please refer to the background slides for the introduction to filters)

In [ ]:
from skimage.filters.rank import mean as mean_filter
from skimage.filters.rank import minimum as min_filter
from skimage.filters.rank import median as median_filter
from skimage.morphology import disk

First, let's examine the effect of filter sizes.

In [ ]:
from skimage.morphology import disk

with sns.axes_style('white'):
    N = 5
    fig, axes = plt.subplots(1, N, figsize=(16, 3))
    for n, ax in enumerate(axes):
        np1 = n + 1
        ax.imshow(np.pad(disk(np1), N-n, 'constant'), interpolation='none')
        c = plt.Circle((np1 + N - n, np1 + N - n), radius=np1, fill=False, lw=4, color='b')
        ax.add_artist(c)
        ax.set_xlim(0, 2 * N + 2)
        ax.set_ylim(0, 2 * N + 2)

Now, try using median filter to remove the salt-pepper noise. Can you see differences in the images?

In [ ]:
f_size = 1
data_filt = median_filter(nodrug_slice["your_fav_protein"], disk(f_size))

max_view = 2000 #change me to change brightness of all images uniformally

fig, ax = plt.subplots(1, 2)#, figsize=
ax[0].imshow(nodrug_slice["your_fav_protein"])
ax[1].imshow(data_filt)
fig.suptitle('Median Filter')

#fig, ax = plt.subplots(1, 2, figsize=(20, 20))
#ax[0, 0].imshow(data_noised, interpolation = 'none', vmin = 0, vmax = max_view)
#ax[0, 1].imshow(data_filt, interpolation = 'none', vmin = 0, vmax = max_view)
#ax[1, 0].imshow(vinc['pax'], interpolation = 'none', vmin = 0, vmax = max_view)
#ax[1, 1].imshow(im_filt, interpolation = 'none', vmin = 0, vmax = max_view)

Now, we can take the filtered image and further pre-process. We'd like to remove the un-even illumination. First, we will extract the background. The radius should be set to at least the size of the largest object that is not part of the background.

In [ ]:
# the radius should be set to at least the size of the largest object that is not part of the background
r = 70
structuring_element = disk(r)
background = min_filter(data_filt, structuring_element)
plt.imshow(background, cmap='inferno', interpolation = 'none')#, vmin = 0, vmax = 3000)

Next, let's substract out the background from the dataset.

In [ ]:
bgs = data_filt - background
bgs[bgs<0] = 0

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(nodrug_slice["your_fav_protein"], interpolation = 'none', vmax = max_val_see)
ax[1].imshow(bgs, interpolation = 'none')

With our pre-processed dataset, let's try the thresholding again.

In [ ]:
### GIVE IT 30 SECONDS TO RUN

from ipywidgets import interactive
@interactive
def show_masks(thresh=(0, data1.max() * 0.3, 500)): #shwo_masks a function, whey thresh have 3 input
    fig, ax = plt.subplots(1, 2, figsize=(64, 50))
    
    mask1 = np.zeros(data1.shape)
    mask2 = np.zeros(data2.shape)
    
    mask1[data1 >=thresh] = 1
    mask2[data2 >=thresh] = 1
    
    ax[0].imshow(mask1, vmin=0, vmax=1)
    ax[1].imshow(mask2, vmin=0, vmax=1)
show_masks

### GIVE IT 30 SECONDS TO RUN


While manual thresholding is convenient, it is not the most reliable way to decide the threshold. How can we systematically and reproducibly define a threshold? Otsu's method looks at the histogram and differentiate "signal" vs "noise" to determine the threshold. We want a value that includes signal only. (please see the slides for more information)

First, we will define a mask function. When a data point is greater than threshold, we will give 1. Otherwise 0.

In [ ]:
#answer
def mask_im(im, threshold):
    mask = np.zeros(im.shape)
    mask[im >=threshold] = 1
    plt.imshow(mask, vmin = 0, vmax = 1)
    return(mask)

Let's use otsu filter from the imported packages.

In [ ]:
#answer:
from skimage.filters import threshold_otsu

data = bgs.copy()
thresh = threshold_otsu(data)
mask = mask_im(data,thresh) #mask_im 
plt.close('all')  # mask_im produces an image of mask, close it for now
fig, ax = plt.subplots(1, 2) #, figsize=(32, 50)) 
ax[0].imshow(mask)#[700:900, :200])
ax[1].imshow(data)#[700:900, :200])

Let's examine what Otsu's method did by looking at the histogram. We can see signal cluster vs noise cluster and the threshold value.

In [ ]:
#answer
sns.distplot(data.flatten(), hist_kws={'log': False}, kde=False) 
plt.axvline(thresh, ls='--', lw=2, c='r', label='Otsu threshold') 
plt.legend()
plt.ylim(0,300000)

We can also extract the threshold value.

In [ ]:
#answer
protein_thresh = threshold_otsu(data) 
print("The calculated masking threshold for the fav_protein is:", protein_thresh)

Exercise: try Otsu's method with the filtered nucleus dataset.

Otsu's method isn't the only way to determine the threshold. A list of various methods are listed here: https://imagej.net/Auto_Threshold Another example is Yen's method, a multi-level thresholding algorithm.

In [ ]:
import skimage.filters as filters
from skimage.morphology import label
protein_mask = mask_im(data, filters.threshold_yen(data)) 
#plt.imshow(label(nuc_mask))
print(label(protein_mask).max()) 

Let's examine istogram for Yen's method.

In [ ]:
from skimage.filters import threshold_yen
protein_thresh_yen = threshold_yen(data) 
print("The calculated masking threshold for the fav_protein is:", protein_thresh)

Can you try 1 other filter?